In [2]:
import pybullet as p

In [3]:
import gym
from gym import spaces

In [4]:
import numpy as np

In [5]:
import time

In [ ]:
import 

In [ ]:
class HumanoidEnv(gym.Env):
    
    '''The gym RL Env class for the Transcend Robot'''
    
    def __init__(self):
        
        self.step = 0
        physicsClient = p.connect(p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())

        # Spawning the Plane in pybullet
        print("LOADING Terrain!")
        z2y = p.getQuaternionFromEuler([-math.pi * 0.5, 0, 0])
        planeId = p.loadURDF("plane_implicit.urdf", [0, 0, 0], z2y, useMaximalCoordinates=True)
        p.configureDebugVisualizer(p.COV_ENABLE_Y_AXIS_UP, 1)
        p.setGravity(0, -9.8, 0)
        p.setPhysicsEngineParameter(numSolverIterations=10)
        p.changeDynamics(planeId, linkIndex=-1, lateralFriction=0.9)
        
        #Spawning the robot in pybullet
        print("LOADING Humanoid!")
        time.sleep(2)
        flags=p.URDF_MAINTAIN_LINK_ORDER+p.URDF_USE_SELF_COLLISION+p.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
        useFixedBase=False
        seld.robotID = p.loadURDF("Transcend.urdf", [0, 1.2, 0], globalScaling=1, useFixedBase=useFixedBase, flags=flags)
        
        #Setting up the camera
        p.resetDebugVisualizerCamera(cameraDistance=1.96, cameraYaw=-90, cameraPitch=-13,cameraTargetPosition=[-0.74, 0.61, -0.01])
        
        #TODO : set friction coefficients between the hands, legs and the surface 
        
        #Setting up my state : essentially all the corresponding joint angles of each joint ID
        self.joint_num = p.getNumJoints(self.robotID)
        self.jointNames = [0]*joint_num
        self.joints_lower_limit = [0]*(self.joint_num)
        self.joints_upper_limit = [0]*(self.joint_num)
        self.joints_present_pos = [0]*(self.joint_num)
        
        for j in range(p.getNumJoints(self.robotID)):
            joint_info = p.getJointInfo(self.robotID, j)
            present_info = p.getJointState(self.robotID, j)
            self.jointNames[j] = joint_info[1]
            self.joints_lower_limit[j] = joint_info[8]
            self.joints_upper_limit[j] = joint_info[9]
            self.joint_present_pos[j] = present_info[0]
            
        #defining my action space
        #defining a normaised action space where all the positions of the joints are mapped from 0 to 1
        self.action_space = spaces.Box(np.array([0]*len(self.jointNames)), np.array([1]*len(self.jointName)))
        
        #defining my observation space
        #defining a normalised observation space where all the positions of the joints are mapped from 0 to 1
        self.observation_space = spaces.Box(np.array([0]*len(self.jointNames)), np.array([1]*len(self.jointName)))
        
        
    def step(self, actions):
        
        self.step = self.step + 1
        rn_actions = [0]*(self.joint_num)
        
        if isinstance(actions, np.ndarray):
            actions = actions.tolist()
            
        #Renormalising the action list to within the actual upper and lower limits of the joint
        for i in range(len(actions)) :
            rn_actions[i] = ((actions[i]-self.joints_upper_limit[i])/(self.joints_upper_limit[i] - self.joints_lower_limit[i]))+1
            
        #Setting joint angles using position control
        p.setJointMotorControlArray(self.robotID, range(38), p.POSITION_CONTROL, rn_actions)
        
        updated_obs = [0]*(p.getNumJoints(self.robotID))
        
        #Getting present angles after change
        for j in range(self.joint_num):
            present_info = p.getJointState(self.robotID, j)
            updated_obs[j] = (present_info[0]-1)*(self.joints_upper_limit[i] - self.joints_lower_limit[i]) + self.joints_upper_limit[i]
            
        
        # TODO : Design reward function
            # Dependent on : distance between both the hands and the surface ( MAYBE CONSIDER JUST THE HIEGHT of the hand )
            # penatly for cost of transportation
            # cost of transportation penalty
            # resultant force torque : Reward ( but maybe with a limit to prevent falling )
            # Huge penalty if there is a collision of any pat with each other or with the floor
        
        
        
        reward = 0
        done = False
        info = {}
        
        return updated_obs, reward, done, info
    
    def reset(self):
        
        #Reseting the simulation
        p.resetBasePositionAndOrientation(self.robotID)
        time.sleep(2)
        
    def close(self):
        p.disconnect()
        
# Defining the class's object variable
env = HumannoidEnv()
        
        
        
        